# DAT210x - Programming with Python for DS

## Module5- Lab10

In [1]:
import numpy as np
import pandas as pd

from sklearn.utils.validation import check_random_state
import scipy.io.wavfile as wavfile

Good Luck! Heh.

### About Audio

Samples are Observations. Each audio file will is a single sample in our dataset.

Find more information about [Audio Samples here](https://en.wikipedia.org/wiki/Sampling_(signal_processing)).

Each .wav file is actually just a bunch of numeric samples, "sampled" from the analog signal. Sampling is a type of discretization. When we mention 'samples', we mean observations. When we mention 'audio samples', we mean the actually "features" of the audio file.

The goal of this lab is to use multi-target, linear regression to generate by extrapolation, the missing portion of the test audio file.

Each one audio_sample features will be the output of an equation, which is a function of the provided portion of the audio_samples:

    missing_samples = f(provided_samples)

You can experiment with how much of the audio you want to chop off and have the computer generate using the Provided_Portion parameter.

Play with this. This is how much of the audio file will be provided, in percent. The remaining percent of the file will be generated via linear extrapolation.

In [19]:
Provided_Portion = 0.25
sample_rate = 8000

### The Assignment

You have to download the dataset (audio files) from the website: https://github.com/Jakobovski/free-spoken-digit-dataset

Start by creating a regular Python List called `zero`:

In [3]:
zero = []

Loop through the dataset and load up all 50 of the `0_jackson*.wav` files using the `wavfile.read()` method: https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.io.wavfile.read.html Be careful! `.read()` returns a tuple and you're only interested in the audio data, and not sample_rate at this point. Inside your for loop, simply append the loaded audio data into your Python list `zero`:

In [4]:
for i in range(0,50):
    w = wavfile.read('Datasets/recordings/0_jackson_'+str(i)+'.wav')
    zero.append(w[1])

Just for a second, convert zero into a DataFrame. When you do so, set the `dtype` to `np.int16`, since the input audio files are 16 bits per sample. If you don't know how to do this, read up on the docs here: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html

Since these audio clips are unfortunately not length-normalized, we're going to have to just hard chop them to all be the same length. Since Pandas would have inserted NANs at any spot to make zero a  perfectly rectangular [n_observed_samples, n_audio_samples] array, do a `dropna` on the Y axis here. Then, convert one back into an NDArray using `yourarrayname.values`:

In [12]:
zero = pd.DataFrame(zero, dtype=np.int16)
print zero.head()
zero.dropna(axis=1,inplace=True) 
print zero.head()
print zero.shape

   0     1     2     3     4     5     6     7     8     9     ...   4077  \
0  -369  -431  -475  -543  -571  -557  -528  -455  -394  -305  ...   2475   
1  -311   -91  -140  -182  -271   -68  -235  -359  -129  -198  ...   -122   
2  -361  -226  -238  -478  -425  -395  -663  -879  -726  -997  ...   -141   
3   323   338   357   388   381   393   391   378   348   313  ...   1323   
4  -302  -312  -103  -305  -145  -339    12  -433  -141  -189  ...    189   

   4078  4079  4080  4081  4082  4083  4084  4085  4086  
0  2500  2331  1786  1057   358   158  -108  -402  -884  
1  -207  -266  -276  -316  -359  -396  -422  -462  -460  
2   -83     0    71   165   284   325   378   416   438  
3  1321  1211   827   292  -244  -779 -1241 -1369 -1229  
4   211   189   134   150   224   265   313   328   327  

[5 rows x 4087 columns]
   0     1     2     3     4     5     6     7     8     9     ...   4077  \
0  -369  -431  -475  -543  -571  -557  -528  -455  -394  -305  ...   2475   
1  -311   

It's important to know how (many audio_samples samples) long the data is now.

`zero` is currently shaped like `[n_samples, n_audio_samples]`, so get the `n_audio_samples` count and store it in a variable called `n_audio_samples`:

In [17]:
#x.dropna(axis=1,inplace=True) 
n_audio_samples = len(zero.columns)
print('n_audio_samples',n_audio_samples)
#x = x.values

#for s in range(0,len(x)):
    
#    print(np.unique(pd.isnull(x.loc[s])))
#    print(len(x.loc[s]))


('n_audio_samples', 4087)


Create your linear regression model here and store it in a variable called `model`. Don't actually train or do anything else with it yet:

In [7]:
from sklearn import linear_model
model = linear_model.LinearRegression()

There are 50 takes of each clip. You want to pull out just one of them, randomly, and that one will NOT be used in the training of your model. In other words, the one file we'll be testing / scoring on will be an unseen sample, independent to the rest of your training set:

In [15]:
# Leave this line alone until you've submitted your lab:
rng = check_random_state(7)
random_idx = rng.randint(zero.shape[0])
print('random_idx: ', random_idx)
test  = zero.loc[random_idx]
#train = np.delete(zero, [random_idx], axis=0)
train = zero.drop(zero.index[random_idx])

('rng: ', <mtrand.RandomState object at 0x11a6dd8c0>)
('random_idx: ', 47)


Print out the shape of `train`, and the shape of `test`.

`train` will be shaped: `[n_samples, n_audio_samples]`, where `n_audio_samples` are the 'features' of the audio file 

`test` will be shaped `[n_audio_features]`, since it is a single sample (audio file, e.g. observation).

In [16]:
print('test: ',test.shape)
print('train: ',train.shape)

0     -258
1     -479
2      577
3     -604
4      296
5     -545
6       43
7      123
8      -74
9      275
10      -6
11      42
12    -135
13      70
14    -751
15     -74
16     -15
17    -949
18    -154
19    -648
20    -188
21     726
22      58
23    -503
24       4
25      46
26      -6
27    -291
28    -633
29    -659
30    -298
31    -604
32     -28
33    -348
34    -358
35    -360
36      68
37      60
38    -546
39    -123
40      11
41     531
42    -967
43      39
44     668
45   -1090
46    -180
47    -706
48     826
49     125
Name: 47, dtype: int16
('test: ', (4087,))
('train: ', (49, 4087))


The test data will have two parts, `X_test` and `y_test`.

`X_test` is going to be the first portion of the test audio file, which we will be providing the computer as input. 

`y_test`, the "label" if you will, is going to be the remaining portion of the audio file. Like such, the computer will use linear regression to derive the missing portion of the sound file based off of the training data its received!

Let's save the original `test` clip, the one you're about to delete half of, to the current directory so that you can compare it to the 'patched' clip once you've generated it. You should have already got the `sample_rate` when you were loading up the .wav files:

In [21]:
wavfile.write('Original Test Clip.wav', sample_rate, test.values)

Prepare the TEST data by creating a slice called `X_test`. It should have `Provided_Portion` * `n_audio_samples` audio sample features, taken from your test audio file, currently stored in variable `test`. In other words, grab the FIRST `Provided_Portion` * `n_audio_samples` audio features from `test` and store it in `X_test`. This should be accomplished using indexing:

In [40]:
x_test = test.loc[:Provided_Portion*n_audio_samples]
y_test = test.loc[Provided_Portion*n_audio_samples:]
print x_test.shape
print y_test.shape
print len(x_test) + len(y_test) == n_audio_samples

(1022,)
(3065,)
True


If the first `Provided_Portion` * `n_audio_samples` features were stored in `X_test`, then we need to also grab the _remaining_ audio features and store them in `y_test`. With the remaining features stored in there, we will be able to R^2 "score" how well our algorithm did in completing the sound file.

In [ ]:
# .. your code here ..

Duplicate the same process for `X_train`, `y_train`. The only differences being:

1. Your will be getting your audio data from `train` instead of from `test`
2. Remember the shape of `train` that you printed out earlier? You want to do this slicing but for ALL samples (observations). For each observation, you want to slice the first `Provided_Portion` * `n_audio_samples` audio features into `X_train`, and the remaining go into `y_test`. All of this should be doable using regular indexing in two lines of code:

In [42]:
x_train = train.loc[:,:Provided_Portion*n_audio_samples]
y_train = train.loc[:,Provided_Portion*n_audio_samples:]
print x_train.shape
print y_train.shape
#print len(x_train) + len(y_train) == n_audio_samples

0      -100
1         9
2      5207
3      9428
4     -6257
5     -9864
6     -2254
7      4893
8      4229
9     -5130
10    -4900
11     9560
12    -8652
13     9797
14    -1827
15     6248
16    -1657
17   -13687
18     -501
19     4334
20    -5091
21    -2653
22     1820
23     6169
24    -1593
25     4437
26     7119
27    -6532
28     6727
29     3284
30     1008
31     2475
32    -1376
33      160
34    -1905
35    -3149
36     -456
37     1821
38     2119
39    -6556
40    -2440
41     -762
42     2593
43     1259
44      416
45      210
46    -1415
48    -1075
49    -3580
Name: 1234, dtype: int16
(49, 1022)
(49, 3065)


SciKit-Learn gets 'angry' if you don't supply your training data in the form of a 2D dataframe shaped like `[n_samples, n_features]`.

So if you only have one SAMPLE, such as is our case with `X_test`, and `y_test`, then by calling `.reshape(1, -1)`, you can turn `[n_features]` into `[1, n_features]` in order to appease SciKit-Learn.

On the other hand, if you only have one FEATURE, you can alternatively call `.reshape(-1, 1)` on your data to turn `[n_samples]` into `[n_samples, 1]`.

Reshape X_test and y_test as [1, n_features]:

In [ ]:
# .. your code here ..

Fit your model using your training data and label:

In [43]:
model.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Use your model to predict the `label` of `X_test`. Store the resulting prediction in a variable called `y_test_prediction`:

In [47]:
y_test_prediction = model.predict(x_test.reshape(1,-1))

/Users/ferrywlto/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


SciKit-Learn will use float64 to generate your predictions so let's take those values back to int16, which is what our .wav files expect:

In [48]:
y_test_prediction = y_test_prediction.astype(dtype=np.int16)

Score how well your prediction would do for some good laughs, by passing in your test data and test label `y_test`:

In [53]:
score = model.score(x_test.reshape(1,-1), y_test.reshape(1,-1))

/Users/ferrywlto/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [64]:
print("Extrapolation R^2 Score: ", score)

('Extrapolation R^2 Score: ', 0.0)


Let's take the first `Provided_Portion` portion of the test clip, the part you fed into your linear regression model. Then, stitch that together with the 'abomination' the predictor model generated for you and then save the completed audio clip:

In [57]:
completed_clip = np.hstack((x_test.reshape(1,-1), y_test_prediction))
wavfile.write('Extrapolated Clip.wav', sample_rate, completed_clip[0])

/Users/ferrywlto/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


Congrats on making it to the end of this crazy lab and module =) !